In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('ratings.csv')
data

,user,item,rating
0,1889878,CC0101EN,3.0
1,1342067,CL0101EN,3.0
2,1990814,ML0120ENv3,3.0
3,380098,BD0211EN,3.0
4,779563,DS0101EN,3.0
...,...,...,...
233301,1540125,DS0101EN,3.0
233302,1250651,PY0101EN,3.0
233303,1003832,CB0105ENv1,3.0
233304,922065,BD0141EN,3.0


In [3]:
user_rating_counts = data['user'].value_counts()
user_rating_counts

user
2          61
5          59
1653994    51
924030     51
746163     50
           ..
1058827     1
715501      1
763737      1
506416      1
711047      1
Name: count, Length: 33901, dtype: int64

In [4]:
users_with_more_than_2_ratings = user_rating_counts[user_rating_counts >= 2].index
users_with_more_than_2_ratings

Index([      2,       5, 1653994,  924030,  746163, 1761291,  382781,  971917,
       1559851, 1445103,
       ...
        435543, 1112795,   86305,  352985,  435122, 1776998,  295083,  481714,
        211118, 1776024],
      dtype='int64', name='user', length=25581)

In [5]:
filtered_data = data[data['user'].isin(users_with_more_than_2_ratings)]

# train_data, test_data = train_test_split(filtered_data, test_size=0.2, random_state=42)

df = filtered_data.pivot_table(index='user', columns='item', values='rating')

# user_item_matrix = train_matrix.fillna(0) 
# user_similarity = cosine_similarity(user_item_matrix)

# user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

In [6]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy


long_df = filtered_data

# Define the Reader for Surprise
reader = Reader(rating_scale=(1, 5))

# Load the data into Surprise format
data = Dataset.load_from_df(long_df[['user', 'item', 'rating']], reader)

# Train-test split (20% of data for testing)
trainset, testset = train_test_split(data, test_size=0.2)

# Use SVD (Singular Value Decomposition) for matrix factorization
model = SVD()

# Train the model on the training set
model.fit(trainset)

# Predict ratings for the test set
test_predictions = model.test(testset)

# Calculate RMSE for the test set predictions
rmse = accuracy.rmse(test_predictions)

# # Predict missing values for all users
# predictions = []
# for user in df.index:
#     user_predictions = []
#     for item in df.columns:
#         prediction = model.predict(user, item)
#         user_predictions.append(prediction.est)
#     predictions.append(user_predictions)

# # Create a DataFrame from the predictions and fill in missing values
# predicted_df = pd.DataFrame(predictions, columns=df.columns, index=df.index)

# # Replace missing values in the original DataFrame with predicted ones
# df_filled = df.copy()
# for user in df.index:
#     for item in df.columns:
#         if pd.isna(df_filled.loc[user, item]):
#             df_filled.loc[user, item] = predicted_df.loc[user, item]

RMSE: 0.0847
